In [10]:
import wandb
import os

In [11]:
RUN_PATH = "team-epoch-iv/detect-harmful-brain-activity/runs/vm5pnb5l"

In [12]:
api = wandb.Api()
run = api.run(f"{RUN_PATH}")

In [13]:
is_sweep_run = run.sweep is not None
print(f"Is sweep run: {is_sweep_run}")

Is sweep run: False


In [14]:
# list all artefacts
model_artefacts = []
for artefact in run.logged_artifacts():
    if artefact.type == "model":
        print(artefact.name)
        model_artefacts.append(artefact)

MultiResidualBiGRU:v353
MultiResidualBiGRU:v354
MultiResidualBiGRU:v357
MultiResidualBiGRU:v358
MultiResidualBiGRU:v361
MultiResidualBiGRU:v362
MultiResidualBiGRU:v365
MultiResidualBiGRU:v366
MultiResidualBiGRU:v369
MultiResidualBiGRU:v370
MultiResidualBiGRU:v374
MultiResidualBiGRU:v376
MultiResidualBiGRU:v379
MultiResidualBiGRU:v382
MultiResidualBiGRU:v385
MultiResidualBiGRU:v386
MultiResidualBiGRU:v389
MultiResidualBiGRU:v392
MultiResidualBiGRU:v395
MultiResidualBiGRU:v396


In [15]:
# download all artefacts
for artefact in model_artefacts:
    for file in artefact.files():
        print(f"Downloading {file.name} from {artefact.name}")
        file.download(root="../downloads")

In [16]:
# get the config type artefact (should be only one)
config_artefact = None
for artefact in run.logged_artifacts():
    if artefact.type == "config":
        config_artefact = artefact
        break

In [17]:
if not is_sweep_run:
    # download the raw, human readable config file
    for file in config_artefact.files():
        if file.name not in ["config.yaml","cv.yaml","train.yaml"]:
            print(f"Downloading {file.name} from {config_artefact.name}")
            file.download(root="../downloads", replace=True)
            # rename the file to the name of the run
            os.rename(f"../downloads/{file.name}", f"../downloads/{run.name}")
            print(f"Renamed to {run.name}")

Renamed to cream-game


In [18]:
from omegaconf import OmegaConf

if is_sweep_run:
    # download the file called config.yaml from the artifact, which includes sweep parameters
    for file in config_artefact.files():
        if file.name == "config.yaml":
            print(f"Downloading {file.name} from {config_artefact.name}")
            file.download(root="../downloads", replace=True)
            
            # read the file into a dictconfig with omegaconf
            config = OmegaConf.load(f"../downloads/{file.name}")        
            config = config.model
            # save the config again
            OmegaConf.save(config, f"../downloads/{file.name}")        
            
            # rename the file to the name of the run
            os.rename(f"../downloads/{file.name}", f"../downloads/{run.name}.yaml")
            print(f"Renamed to {run.name}.yaml")
